In [14]:
import pandas as pd
import numpy as np
from astropy.convolution import convolve, Box1DKernel
import bokeh
from bokeh.layouts import column, row
from bokeh.transform import factor_mark, factor_cmap
from bokeh.models import ColumnDataSource, Slider, TextInput, RadioButtonGroup, Button, LinearColorMapper
from bokeh.events import DoubleTap
from bokeh.plotting import figure

from bokeh.io import curdoc, output_notebook, show

output_notebook()

Loading BokehJS ...

In [15]:
# Forked from echelle module
def echelle(freq, power, dnu, fmin=0.0, fmax=None, offset=0.0, sampling=0.1):
    """Calculates the echelle diagram. Use this function if you want to do
    some more custom plotting.

    Parameters
    ----------
    freq : array-like
        Frequency values
    power : array-like
        Power values for every frequency
    dnu : float
        Value of deltanu
    fmin : float, optional
        Minimum frequency to calculate the echelle at, by default 0.
    fmax : float, optional
        Maximum frequency to calculate the echelle at. If none is supplied,
        will default to the maximum frequency passed in `freq`, by default None
    offset : float, optional
        An offset to apply to the echelle diagram, by default 0.0

    Returns
    -------
    array-like
        The x, y, and z values of the echelle diagram.
    """
    if fmax == None:
        fmax = freq[-1]

    # Apply offset
    fmin = fmin - offset
    fmax = fmax - offset
    freq = freq - offset

    # Quality of life checks
    if fmin <= 0.0:
        fmin = 0.0
    else:
        # Make sure it partitions exactly
        fmin = fmin - (fmin % dnu)

    # trim data
    index = (freq >= fmin) & (freq <= fmax)
    trimx = freq[index]

    # median interval width
    samplinginterval = np.median(trimx[1:-1] - trimx[0:-2]) * sampling

    # Fixed sampling interval x values
    xp = np.arange(fmin, fmax + dnu, samplinginterval)

    # Interpolant (approximation) for xp values (given the frequency and power)
    yp = np.interp(xp, freq, power)

    # Number of stacks and Number of elements in each stack
    n_stack = int((fmax - fmin) / dnu)
    n_element = int(dnu / samplinginterval)

    # Number of rows for each datapoint (elongate graph)
    morerow = 2

    # Array of length = number of stacks
    arr = np.arange(1, n_stack) * dnu

    # Double the size (due to 2 rows?)
    arr2 = np.array([arr, arr])

    # y-values of each stack - reshape 2 stacks
    yn = np.reshape(arr2, len(arr) * 2, order="F")

    # Ending values - Insert 0 in beginning and append number of stacks * dnu, plus offsets
    yn = np.insert(yn, 0, 0.0)
    yn = np.append(yn, n_stack * dnu) + fmin + offset

    # x values of partition
    xn = np.arange(1, n_element + 1) / n_element * dnu

    # image as 2D array
    z = np.zeros([n_stack * morerow, n_element])

    # Add yp values to rows of image
    for i in range(n_stack):
        for j in range(i * morerow, (i + 1) * morerow):
            # Multiple rows of the same data
            z[j, :] = yp[n_element * (i) : n_element * (i + 1)]

    return xn, yn, z

In [19]:
def bokeh_app(doc):
    #===========================================================
    # Data
    #===========================================================
    # Read in csv
    ps_df = pd.read_csv("data/HD47205_PS.csv")

    # Prepare numpy array data
    freq = ps_df.freq.to_numpy()
    pows = ps_df.pows.to_numpy()

    # Constants we already know about the star
    Dnu = 13.65
    numax = 178

    # Magic maths
    amp = np.sqrt(pows)
    df=freq[1]-freq[0]
    smooth = .05/df  # in muHz
    amp=convolve(amp, Box1DKernel(smooth))

    # x-ticks, y-ticks, and 2d array z as image
    xn, yn, z = echelle(freq, amp, Dnu, 
        fmin=max(numax - 100, 0), fmax=numax + 100)

    # echelle p
    # ixn, iyn, iz = echelle(1/freq, amp, 1/Dnu, fmin=1/(numax + 100), fmax=1/max(numax - 100, 0))


    # Data source for echelle
    source = ColumnDataSource(data=dict(z=[z]))

    # Tuple of (x,y) for l node labelling
    l_points = ColumnDataSource(data=dict(x=[], y=[], ix=[], iy=[], l=[]))

    #===========================================================
    # Plot setup
    #===========================================================

    # Tools and tooltips
    tools = "box_select,save"
    tooltips = [("x", "$x"), ("y", "$y"), ("value", "@image")]# What is displayed

    # Dimensions of figure
    width=480
    height=640

    # Create plots
    echelle_nu = figure(tooltips=tooltips,
                tools=tools,
                plot_width=width, plot_height=height)
    echelle_nu.x_range.range_padding = echelle_nu.y_range.range_padding = 0


    """
    echelle_p = figure(tooltips=tooltips,
                tools=tools + ",wheel_zoom",
                plot_width=width, plot_height=height)
    echelle_p.x_range.range_padding = echelle_p.y_range.range_padding = 0
    """

    # More intense should have more color
    palette = list(bokeh.palettes.Blues256)
    palette.reverse()


    # output_file("image.html", title="image.py example")
    # bokeh.io.show(p, inputs)  # open a browser


    #===========================================================
    # Data update
    #===========================================================

    # Selections for l modes
    l_modes = RadioButtonGroup(labels=["l=0", "l=1", "l=2"], active=0, aspect_ratio="auto")

    # SLider for Dnu guesses
    partition = Slider(title="Dnu (mu Hz)", value=Dnu, start=Dnu - 5, end=Dnu + 5, step=0.1)

    # Button to undo
    undo_label = Button(label="Remove last label", button_type="success")

    def update_data(attr, old, new):
        """
        Update echelle diagram
        """
        # Get current slider values
        width = partition.value

        # Get new echelle
        xn, yn, z = echelle(freq, amp, width, 
            fmin=max(numax - 100, 0), fmax=numax + 100)

        # echelle p
        # ixn, iyn, iz = echelle(1/freq, amp, 1/width, fmin=1/(numax + 100), fmax=1/max(numax - 100, 0))

        # Update to source
        source.data.update(dict(z=[z]))

        print(f"Dnu changed to {new}")


    def add_point(event):
        """
        Add a l mode label by double clicking
        """
        # Get points upon double click
        x = event.x
        y = event.y

        # Add points to mode's data and update
        x_vals = l_points.data['x']
        y_vals = l_points.data['y']
        ix_vals = l_points.data['ix']
        iy_vals = l_points.data['iy']
        l_vals = l_points.data['l']
        l = str(l_modes.active)

        # Add to list and update
        x_vals.append(x)
        y_vals.append(y)
        ix_vals.append(1/x)
        iy_vals.append(1/y)    
        l_vals.append(f"l={l}")

        l_points.data.update(dict(x=x_vals, y=y_vals, 
            ix=ix_vals, iy=iy_vals, l=l_vals))

        print(f"Added ({x:.1f}, {y:.1f}) and ({1/x:.1f}, {1/y:.1f}) to l={l} mode")


    def remove_point(event):
        """
        Remove the last label
        """
        x_vals = l_points.data['x']
        ix_vals = l_points.data['ix']
        iy_vals = l_points.data['iy']    
        y_vals = l_points.data['y']
        l_vals = l_points.data['l']

        if len(x_vals) > 0:
            x = x_vals.pop(-1)
            y = y_vals.pop(-1)
            ix = ix_vals.pop(-1)
            iy = iy_vals.pop(-1)
            l = l_vals.pop(-1)
            print(f"Removed ({x:.1f}, {y:.1f}) to l={l} mode")

            l_points.data.update(dict(x=x_vals, y=y_vals, 
                ix=ix_vals, iy=iy_vals, l=l_vals))


    # Interactive changes
    partition.on_change('value', update_data)
    echelle_nu.on_event(DoubleTap, add_point)
    undo_label.on_click(remove_point)

    inputs = column(partition)

    # This is a terrible hack and I hate Bokeh
    # warnings.simplefilter("ignore", BokehUserWarning)

    #===========================================================
    # Finalize
    #===========================================================

    # must give a vector of images
    echelle_nu.image(image='z', x=xn.min(), y=yn.min(), 
        dw=xn.max()-xn.min(), dh=yn.max()-yn.min(), 
        palette=palette, level="image", source=source)
    echelle_nu.xgrid.visible = echelle_nu.ygrid.visible = False

    """
    echelle_p.image(image='iz', x=ixn.max(), y=iyn.max(),
        dw=ixn.max()-ixn.min(), dh=iyn.max()-iyn.min(), 
        palette=palette, level="image", source=source)
    echelle_p.xgrid.visible = echelle_nu.ygrid.visible = False
    echelle_p.x_range.flipped = echelle_p.y_range.flipped = True
    """

    # Mode labels and styles
    L_MODES = ['l=0', 'l=1', 'l=2']
    MARKERS = ['square', 'circle', 'triangle']

    echelle_nu.scatter(x='x', y='y', source=l_points, legend_field='l',
        size=18, fill_alpha=1,
        marker=factor_mark('l', MARKERS, L_MODES),
        color=factor_cmap('l', 'Category10_3', L_MODES))

    """
    echelle_p.scatter(x='ix', y='iy', source=l_points, legend_field='l',
        size=18, fill_alpha=1,
        marker=factor_mark('l', MARKERS, L_MODES),
        color=factor_cmap('l', 'Category10_3', L_MODES))
    """

    """
    curdoc().add_root(column(row(echelle_nu, echelle_p), inputs, row(l_modes, undo_label)))
    curdoc().title = "Sliders"
    """
    doc.add_root(column(row(echelle_nu), inputs, row(l_modes, undo_label)))
    doc.title = "Interactive Echelle Labelling"
    

In [21]:
show(bokeh_app)

Added (11.6, 223.7) and (0.1, 0.0) to l=0 mode
Added (11.2, 181.6) and (0.1, 0.0) to l=0 mode
Added (12.9, 102.1) and (0.1, 0.0) to l=0 mode
Added (10.9, 86.8) and (0.1, 0.0) to l=0 mode
Added (8.0, 155.9) and (0.1, 0.0) to l=0 mode
Added (11.3, 134.2) and (0.1, 0.0) to l=0 mode
Added (8.3, 104.8) and (0.1, 0.0) to l=0 mode
Added (2.8, 201.7) and (0.4, 0.0) to l=1 mode
Added (2.3, 180.6) and (0.4, 0.0) to l=1 mode
Added (2.2, 145.9) and (0.5, 0.0) to l=1 mode
Added (4.2, 199.0) and (0.2, 0.0) to l=2 mode
Added (4.2, 168.6) and (0.2, 0.0) to l=2 mode
Added (4.4, 143.6) and (0.2, 0.0) to l=2 mode
Added (4.5, 223.7) and (0.2, 0.0) to l=2 mode
Added (3.0, 222.4) and (0.3, 0.0) to l=1 mode
